In [1]:
import requests
import pandas as pd
import requests
import numpy as np
import concurrent.futures


In [2]:
# excel_file = r"D:\Onedrive\OneDrive - UNI CONSULTING\Attachments\Code\uni solr\uni_solr\classification\ko_prod_class_22-10_with_imglink.xlsx"  # Đường dẫn đến file Excel của bạn
excel_file =  "../threads/all_links_18022025.xlsx"

In [3]:
df = pd.read_excel(excel_file)


In [1]:
df.head()

NameError: name 'df' is not defined

In [8]:
df.iloc[1,8]

"['/clip/framework/filedownload/kcs4gImageDownload.do?attchFileId=PCA-20241231-000060152663SL32']"

In [ ]:
# df['Uploaded Links'] = ""

In [22]:
import os

folder_path = r"D:\Onedrive\OneDrive - UNI CONSULTING\Eximhow_img\18022025"  # Đường dẫn tới folder

# Lấy danh sách tất cả file trong folder
downloaded_file_names = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

In [ ]:
# Hàm tải ảnh từ URL về máy cục bộ
def download_image(url):
    # Lấy tên file từ URL
    url = url.strip("'\"")  # Loại bỏ ký tự thừa ở đầu và cuối
    file_name = url.split("attchFileId=")[-1]  # Trích xuất ID từ URL
    file_name = f"{folder_path}\{file_name}.jpg"  # Thêm phần mở rộng file
    if f"{file_name}.jpg" not in downloaded_file_names:
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status()  # Kiểm tra lỗi HTTP

            with open(file_name, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            print("Đã tải ", file_name)
            return file_name
        except Exception as e:
            print(f"Lỗi tải ảnh: {e}")
            return None

# Hàm tải ảnh và cập nhật `Uploaded Links` cho mỗi hàng
def process_row(index, row):
    uploaded_links = []
    
    if row[7] >= 1 and (pd.isna(row[10]) or not row[10] or "lỗi" in row[10]):
        image_links = row[8]
        
        if pd.notna(image_links):  # Kiểm tra ô không rỗng
            image_links = image_links.strip('][').split(', ')
            if image_links == ['']:
                return index, ''  # Nếu không có link ảnh trong hàng đó
            else:
                for link in image_links:
                    try:
                        link = link.strip("'")
                        full_path_link = "https://unipass.customs.go.kr" + link
                        local_filename = download_image(full_path_link)  # Hàm tải ảnh về
                        onlinepath = "https://hs-search-img-pdf.s3.ap-southeast-1.amazonaws.com/Eximhow_img/18022025/"  # Thay bằng đường dẫn chính xác
                        if local_filename:
                            uploaded_links.append(onlinepath + local_filename)
                        else:
                            uploaded_links.append("lỗi")  # Đánh dấu lỗi vào danh sách nếu gặp lỗi
                    except Exception as e:
                        print(f"Lỗi khi tải ảnh từ {link}: {e}")
                        uploaded_links.append("lỗi")  # Đánh dấu lỗi vào danh sách nếu gặp lỗi

                return index, ", ".join(uploaded_links)  # Gộp các liên kết đã tải lên thành chuỗi

    return index, row[10]  # Nếu `image_links` rỗng hoặc không tải được ảnh,trả về giá trị cũ

# Hàm để xử lý mỗi phần của DataFrame với các hàng nhất định
def process_dataframe_part(part_df):
    results = []
    for index, row in part_df.iterrows():
        result = process_row(index, row)
        results.append(result)
    return results


<string>:6: SyntaxWarning: invalid escape sequence '\{'
<>:6: SyntaxWarning: invalid escape sequence '\{'
<string>:6: SyntaxWarning: invalid escape sequence '\{'
<>:6: SyntaxWarning: invalid escape sequence '\{'
C:\Users\nguye\AppData\Local\Temp\ipykernel_15212\1321253653.py:6: SyntaxWarning: invalid escape sequence '\{'
  file_name = f"{folder_path}\{file_name}.jpg"  # Thêm phần mở rộng file


In [27]:
# Chia DataFrame thành 50 phần nhỏ
num_threads = 50
df_split = np.array_split(df, num_threads)

# Khởi tạo ThreadPoolExecutor với 50 thread
with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
    # Xử lý từng phần của DataFrame song song
    futures = [executor.submit(process_dataframe_part, part_df) for part_df in df_split]

    # Cập nhật lại DataFrame với kết quả đã xử lý
    for future in concurrent.futures.as_completed(futures):
        results = future.result()
        for index, uploaded_links in results:
            df.at[index, 'Uploaded Links'] = uploaded_links  # Cập nhật lại cột 'Uploaded Links'

print("Hoàn tất xử lý đa luồng.")

c:\Users\nguye\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
C:\Users\nguye\AppData\Local\Temp\ipykernel_15212\1321253653.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[7] >= 1 and (pd.isna(row[10]) or not row[10] or "lỗi" in row[10]):
C:\Users\nguye\AppData\Local\Temp\ipykernel_15212\1321253653.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  image_links = row[8]


Đã tải  D:\Onedrive\OneDrive - UNI CONSULTING\Eximhow_img\18022025\PCA-20241115-00005958906Ek2gZ.jpg
Đã tải  D:\Onedrive\OneDrive - UNI CONSULTING\Eximhow_img\18022025\PCA-20241212-00006003289hVc5c.jpg
Đã tải  D:\Onedrive\OneDrive - UNI CONSULTING\Eximhow_img\18022025\PCA-20241031-00005935584z3W4n.jpg
Đã tải  D:\Onedrive\OneDrive - UNI CONSULTING\Eximhow_img\18022025\PCA-20241209-00005997070cXjLS.jpg
Đã tải  D:\Onedrive\OneDrive - UNI CONSULTING\Eximhow_img\18022025\PCA-20250117-00006029943tJkvH.jpg
Đã tải  D:\Onedrive\OneDrive - UNI CONSULTING\Eximhow_img\18022025\PCA-20241108-00005948655umyEY.jpg
Đã tải  D:\Onedrive\OneDrive - UNI CONSULTING\Eximhow_img\18022025\PCA-20241230-00006013500YaZhS.jpg
Đã tải  D:\Onedrive\OneDrive - UNI CONSULTING\Eximhow_img\18022025\PCA-20241213-00006003825yLI5G.jpg
Đã tải  D:\Onedrive\OneDrive - UNI CONSULTING\Eximhow_img\18022025\PCA-20241209-00005996951vu1BT.jpg
Đã tải  D:\Onedrive\OneDrive - UNI CONSULTING\Eximhow_img\18022025\PCA-20241216-00006004943

C:\Users\nguye\AppData\Local\Temp\ipykernel_15212\1321253653.py:49: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return index, row[10]  # Nếu `image_links` rỗng hoặc không tải được ảnh,trả về giá trị cũ


Đã tải  D:\Onedrive\OneDrive - UNI CONSULTING\Eximhow_img\18022025\PCA-20250110-000060255195BMJo.jpg
Đã tải  D:\Onedrive\OneDrive - UNI CONSULTING\Eximhow_img\18022025\PCA-20241224-00006010835zbWYT.jpg
Đã tải  D:\Onedrive\OneDrive - UNI CONSULTING\Eximhow_img\18022025\PCA-20241129-00005982270fD6QT.jpg
Đã tải  D:\Onedrive\OneDrive - UNI CONSULTING\Eximhow_img\18022025\PCA-20250114-00006027510HCNxy.jpg
Đã tải  D:\Onedrive\OneDrive - UNI CONSULTING\Eximhow_img\18022025\PCA-20241219-00006008962pzJWu.jpg
Đã tải  D:\Onedrive\OneDrive - UNI CONSULTING\Eximhow_img\18022025\PCA-20241219-00006008429GpIOE.jpg
Đã tải  D:\Onedrive\OneDrive - UNI CONSULTING\Eximhow_img\18022025\PCA-20241219-00006008736XX7aN.jpg
Đã tải  D:\Onedrive\OneDrive - UNI CONSULTING\Eximhow_img\18022025\PCA-20250117-00006029944XhbCV.jpg
Đã tải  D:\Onedrive\OneDrive - UNI CONSULTING\Eximhow_img\18022025\PCA-20241224-00006010619bjqkp.jpg
Đã tải  D:\Onedrive\OneDrive - UNI CONSULTING\Eximhow_img\18022025\PCA-20241118-00005962846

In [28]:
len (df)

294

In [29]:
# Ghi lại DataFrame vào file Excel
df.to_excel(r"D:\Onedrive\OneDrive - UNI CONSULTING\Attachments\Code\uni_craw_tvpl\uni_craw_korea\threads\downloaded_img.xlsx", index=False)